# LoRa variability static devices data cleanup

In [1]:
import pandas as pd
import numexpr
%matplotlib inline

The packet data are read from the CSV file.
They are then joined with the timestamp of the first valid packet per device.

In [2]:
loravar = pd.read_csv('../data/lora_mons_static.csv')
devices = pd.read_csv('../data/devices.csv')
loravar = loravar.merge(devices, on='dev_id')
loravar.shape

(41896, 15)

In [3]:
loravar.dtypes

received            object
dev_id              object
dev_eui             object
gtw_id              object
counter              int64
frequency          float64
data_rate           object
coding_rate         object
rssi                 int64
snr                float64
battery            float64
humidity           float64
pressure           float64
temperature        float64
start_timestamp     object
dtype: object

Data before the first valid timestamp must be dropped.

In [4]:
loravar['received'] = pd.to_datetime(loravar['received'])
loravar['start_timestamp'] = pd.to_datetime(loravar['start_timestamp'])
loravar = loravar.loc[loravar['received'] > loravar['start_timestamp']]

Some columns are redundant and can be dropped.

In [5]:
loravar.drop(columns=['dev_eui', 'counter', 'coding_rate', 'start_timestamp'], inplace=True)
loravar.shape

(38780, 11)

The dataset contains a few incomplete rows that we can drop.
These rows miss the data coming from the payload.

In [6]:
pd.isna(loravar).sum()

received        0
dev_id          0
gtw_id          0
frequency       0
data_rate       0
rssi            0
snr             0
battery        63
humidity       63
pressure       63
temperature    63
dtype: int64

In [7]:
loravar.dropna(inplace=True)
loravar.shape

(38717, 11)

Since they were fixed in the database, there is no duplicate to drop.

In [8]:
loravar.duplicated().sum()

0

After cleanup, the dataset covers the time range hereunder.
Although the situation might be very different per device and per gateway
As could be expected, the gateway on the roof of Cité Houzeau is the one that received the most packets.

In [9]:
loravar[['received']].max() - loravar[['received']].min()

received   41 days 02:20:25.362269
dtype: timedelta64[ns]

In [10]:
loravar.groupby(['dev_id', 'gtw_id'])['received'].count().unstack(fill_value=0)

gtw_id,eui-0000024b08030186,iotlab-rpi-03
dev_id,,
static_6_01,2197,0
static_6_02,3053,0
static_6_03,5659,3483
static_7_01,5555,4551
static_7_02,5557,77
static_8_01,4219,0
static_8_03,2623,1723
static_8_04,20,0


In [11]:
loravar.to_pickle('../data/lora_mons_static_clean.pkl.gz', compression='gzip')